In [1]:
import torch.nn.functional as F
import torch
import os
import numpy as np
import pandas as pd

In [ ]:
def pad_or_crop(grid, target_shape=(150, 150, 150)):
    """Pads or crops a 3D occupancy grid to match target shape."""
    grid_tensor = torch.tensor(grid, dtype=torch.float32)
    current_shape = grid_tensor.shape

    # Pad if smaller
    pad_sizes = [(max(0, t - c) // 2, max(0, t - c) - max(0, t - c) // 2) for t, c in zip(target_shape, current_shape)]
    grid_tensor = F.pad(grid_tensor, [pad_sizes[2][0], pad_sizes[2][1],  # Z-axis
                                      pad_sizes[1][0], pad_sizes[1][1],  # Y-axis
                                      pad_sizes[0][0], pad_sizes[0][1]]) # X-axis

    # Crop if larger
    if grid_tensor.shape[0] > target_shape[0]:
        start_x = (grid_tensor.shape[0] - target_shape[0]) // 2
        grid_tensor = grid_tensor[start_x:start_x+target_shape[0], :, :]
    if grid_tensor.shape[1] > target_shape[1]:
        start_y = (grid_tensor.shape[1] - target_shape[1]) // 2
        grid_tensor = grid_tensor[:, start_y:start_y+target_shape[1], :]
    if grid_tensor.shape[2] > target_shape[2]:
        start_z = (grid_tensor.shape[2] - target_shape[2]) // 2
        grid_tensor = grid_tensor[:, :, start_z:start_z+target_shape[2]]

    return grid_tensor.numpy()

In [ ]:
input_folder = "/Users/User/Documents/Project_JEB/JEB_Data/SimJEB_Occupancy_Grids/"  # Folder containing original .npy files
output_folder = "/Users/User/Documents/Project_JEB/JEB_Data/processed_grids/"

# Ensure output folder exists
os.makedirs(output_folder, exist_ok=True)

# Process all .npy files
for filename in os.listdir(input_folder):
    if filename.endswith(".npy"):
        grid = np.load(os.path.join(input_folder, filename))
        resized_grid = pad_or_crop(grid, target_shape=(150, 150, 150))  # Change method if needed
        np.save(os.path.join(output_folder, filename), resized_grid)

print("All occupancy grids have been standardized!")

In [ ]:
df = pd.read_csv("/Users/User/Documents/Project_JEB/JEB_Data/cleaned_workingdata.csv")
# Path to occupancy grid files
grid_folder = "/Users/User/Documents/Project_JEB/JEB_Data/processed_grids/"
grid_files = sorted([f for f in os.listdir(grid_folder) if f.endswith(".npy")])

# Match files to dataset IDs
df["file_name"] = df["id"].astype(str) + ".npy"

# Verify matching
print(df[["id", "file_name", "max_stress"]].head())
print(f"Number of grid files: {len(grid_files)}, Dataset samples: {len(df)}")